# NVIDIA Blueprint: Retail Shopping Assistant

 This notebook will deploy the **Retail Shopping Assistant** AI Blueprint. You will install the necessary prerequisites, spin up the NVIDIA NIM™ microservices (either locally or using cloud endpoints), and deploy the multi-agent retail shopping system. Once deployed, you will have a fully functional reference UI as well as sample code which you can use to build intelligent retail shopping experiences with features like product search, cart management, visual search, and conversational AI interactions.

## Diagram
![Shopping Assistant Diagram](shopping-assistant-diagram.png)

# Getting Started
>[Prerequisites](#Prerequisites)  
>[Environment Setup](#Environment-Setup)  
>[Configuration Options](#Configuration-Options)  
>[Deployment Options](#Deployment-Options)  
>[Local NIM Deployment](#Local-NIM-Deployment)  
>[Cloud NIM Deployment](#Cloud-NIM-Deployment)  
>[Application Deployment](#Application-Deployment)  
>[Validate Deployment](#Validate-Deployment)  
>[API Reference](#API-Reference)  
>[Next Steps](#Next-Steps)  
>[Shutting Down Blueprint](#Stopping-Services-and-Cleaning-Up)  
>[Appendix](#Appendix)  
________________________

## Prerequisites

### Clone repository and install software

1. **Clone** Retail Shopping Assistant Git repository

In [ ]:
#!git clone https://github.com/NVIDIA-AI-Blueprints/retail-shopping-assistant.git #Using HTTP
!git clone git@github.com:NVIDIA-AI-Blueprints/retail-shopping-assistant.git #Using SSH
!cd retail-shopping-assistant

2. Install **[Docker](https://docs.docker.com/engine/install/ubuntu/)**

<div class="alert alert-block alert-success">
    <b>Tip:</b> Ensure the Docker Compose plugin version is 2.29.1 or higher.  Run `docker compose version` to confirm. Refer to Install the Compose plugin Docker documentation for more information.
</div>

3. Install **[NVIDIA Container Toolkit](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/latest/install-guide.html#installing-the-nvidia-container-toolkit)** to configure Docker for GPU-accelerated containers, for example Milvus, NVIDIA NIM.
 If you are using a system deployed with Brev you can skip this step since Brev systems come with NVIDIA Container Toolkit preinstalled. 

<div class="alert alert-block alert-info">
    <b>Note:</b> After installing the toolkit, follow the instructions in the Configure Docker section in the NVIDIA Container Toolkit documentation.
</div>

### System Requirements

#### For Local NIM Deployment (Recommended)
- **GPUs**: 4x H100 GPUs (or equivalent)
- **RAM**: 128GB+ system memory
- **Storage**: 100GB+ available disk space
- **CPU**: 16+ cores

#### For Cloud NIM Deployment
- **RAM**: 32GB+ system memory
- **Storage**: 50GB+ available disk space
- **CPU**: 8+ cores
- **Network**: Stable internet connection

<div class="alert alert-block alert-info">
    <b>Note:</b> Use oauthtoken as the username and your API key as the password. The $oauthtoken username is a special name that indicates that you will authenticate with an API key and not a user name and password.
</div>

### Get API Keys

#### Let's set the NVIDIA API Catalog key.

This NVIDIA API Catalog key will be used to access cloud hosted models in API Catalog.

You can use different model API endpoints with the same API key.

1. Navigate to **[NVIDIA API Catalog](https://build.nvidia.com/explore/discover)**.

2. Select a model, such as llama3-8b-instruct.

3. Select an **Input** option. The following example is of a model that offers a Docker option. Not all of the models offer this option, but all include a "Get API Key" link

4. Click **Get API Key**.

5. Select **"Generate Key"**

6. **Copy your key** and store it in a secure place. Do not share it.

<div class="alert alert-block alert-success">
    <b>Tip:</b> The key begins with the letters nvapi-.
</div>

7. Input your key to the local environment using the following code block.

In [11]:
import getpass
import os
if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key

#### Let's log into the NVIDIA Container Registry.
 
The NVIDIA NGC API Key is a mandatory key that is required to use this blueprint. This is needed to log into the NVIDIA container registry, nvcr.io, and to pull secure container images used in this NVIDIA NIM Blueprint.
Refer to [Generating NGC API Keys](https://docs.nvidia.com/ngc/gpu-cloud/ngc-user-guide/index.html#generating-api-key) in the NVIDIA NGC User Guide for more information.

Authenticate with the NVIDIA Container Registry with the following commands:

In [12]:
!docker login nvcr.io --username '$oauthtoken' --password $NVIDIA_API_KEY

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded


## Environment Setup

Next we will set up the environment variables and create necessary directories. Here, each enfironment variable controls a different model container within the model. By default we set all of these to use our NVIDIA_API_KEY, but you can use different keys and different endpoints for your embedding model, LLM, and guardrails. 

In [13]:
# Set environment variables
os.environ["NGC_API_KEY"] = os.environ.get("NVIDIA_API_KEY", "")
os.environ["LLM_API_KEY"] = os.environ["NGC_API_KEY"]
os.environ["EMBED_API_KEY"] = os.environ["NGC_API_KEY"]
os.environ["RAIL_API_KEY"] = os.environ["NGC_API_KEY"]

# Create NIM cache directory
local_nim_cache = os.path.expanduser("~/.cache/nim")
os.environ["LOCAL_NIM_CACHE"] = local_nim_cache
os.makedirs(local_nim_cache, exist_ok=True)
os.system(f"chmod a+w {local_nim_cache}")

print(f"Environment variables set")
print(f"NIM cache directory: {local_nim_cache}")

Environment variables set
NIM cache directory: /home/nvidia/.cache/nim


## Configuration Options

The Retail Shopping Assistant supports a flexible configuration override system that allows you to switch between different deployment scenarios without modifying the base configuration files.

### How Configuration Override Works

1. **Base Configuration**: The application loads the base `config.yaml` file
2. **Override Detection**: If the `CONFIG_OVERRIDE` environment variable is set, the system looks for an override file
3. **Merge Process**: The override file values are merged into the base configuration, with override values taking precedence

### Available Configuration Options

| Configuration | Environment Variable | Description |
|---------------|---------------------|-------------|
| Local NIMs | `CONFIG_OVERRIDE=config-local.yaml` | Use locally deployed NIMs |
| Cloud NIMs | `CONFIG_OVERRIDE=config-build.yaml` | Use NVIDIA API Catalog hosted endpoints |
| Custom | `CONFIG_OVERRIDE=config-custom.yaml` | Use your own custom configuration |
| Default | (not set) | Use base configuration only |

## Deployment Options

The Retail Shopping Assistant supports two deployment options:

### Option 1: Local NIM Deployment (Recommended)
- **Best for**: Development, testing, production with GPU resources
- **Requirements**: 4x H100 GPUs, 128GB+ RAM
- **Pros**: Maximum performance, complete privacy, no ongoing cloud costs
- **Cons**: Higher hardware investment, more complex setup

### Option 2: Cloud NIM Deployment
- **Best for**: Development, testing, production without local GPUs
- **Requirements**: Stable internet connection
- **Pros**: No local GPU requirements, faster setup, pay-per-use
- **Cons**: Ongoing cloud costs, network latency, API rate limits

Choose your deployment option below:

## Local NIM Deployment

### Step 1: Set Configuration for Local NIMs

In [14]:
# Set configuration override for local NIM deployment
os.environ["CONFIG_OVERRIDE"] = "config-local.yaml"
print(f"Configuration override set to: {os.environ['CONFIG_OVERRIDE']}")

Configuration override set to: config-local.yaml


### Step 2: Verify GPU Setup

In [15]:
# Check NVIDIA drivers and GPU availability
!nvidia-smi

Thu Jul 24 13:22:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 NVL                On  |   00000000:2E:00.0 Off |                    0 |
| N/A   36C    P0             63W /  400W |       1MiB /  95830MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

### Step 3: Deploy Local NIMs

This will start the following NVIDIA NIM microservices:
- **Llama 3.1 70B Instruct**: Large language model for conversation
- **NV-CLIP**: Visual understanding model for image search
- **NV-EmbedQA E5-v5**: Embedding model for semantic search
- **Llama 3.1 Nemoguard Content Safety**: Content moderation
- **Llama 3.1 Nemoguard Topic Control**: Topic control and safety

In [18]:
# Start local NIMs
!cd .. && docker compose -f docker-compose-nim-local.yaml up -d

WARN[0000] /home/nvidia/cbrissette/retail-shopping-assistant/docker-compose-nim-local.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
[+] Running 1/5
 ⠋ Container retail-shopping-assistant-embedqa-1        Creating           0.1s 
 ⠋ Container retail-shopping-assistant-content-1        Creating           0.1s 
 ⠋ Container retail-shopping-assistant-topic_control-1  Creating           0.1s 
 ✔ Container retail-shopping-assistant-llama-1          Created            0.1s 
 ⠋ Container retail-shopping-assistant-nvclip-1         Creating           0.1s 
[+] Running 0/5
 ⠙ Container retail-shopping-assistant-embedqa-1        Starting           0.2s 
 ⠙ Container retail-shopping-assistant-content-1        Starting           0.2s 
 ⠙ Container retail-shopping-assistant-topic_control-1  Starting           0.2s 
 ⠙ Container retail-shopping-assistant-llama-1          Starting           0.2s 
 ⠙ Container retail-shopping-assistant-nvcl

<div class="alert alert-block alert-info">
    <b>Note:</b> This may take up to **15 minutes** to complete as it downloads and initializes the NIM models. The below block will monitor our progress downloading and launching the largest of our models (Llama 3.1 70B) for twn minutes. This may take a different amount of time depending on your system. Feel free to change the timeout.
</div>

In [19]:
# Monitor NIM startup progress
!cd .. && timeout 600 docker logs -f retail-shopping-assistant-llama-1


== NVIDIA Inference Microservice LLM NIM ==

NVIDIA Inference Microservice LLM NIM Version 1.3.3
Model: meta/llama-3.1-70b-instruct

Container image Copyright (c) 2016-2024, NVIDIA CORPORATION & AFFILIATES. All rights reserved.

The NIM container is governed by the NVIDIA Software License Agreement (found at https://www.nvidia.com/en-us/agreements/enterprise-software/nvidia-software-license-agreement) and the Product Specific Terms for AI Products (found at https://www.nvidia.com/en-us/agreements/enterprise-software/product-specific-terms-for-ai-products).

A copy of this license can be found under /opt/nim/LICENSE.

The use of this model is governed by the NVIDIA AI Foundation Models Community License Agreement (https://www.nvidia.com/en-us/agreements/enterprise-software/nvidia-ai-foundation-models-community-license-agreement).

ADDITIONAL INFORMATION: Llama 3.1 Community License Agreement, Built with Llama.

You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` 

### Step 4: Verify NIM Deployment

In [22]:
# Check NIM container status
!cd .. && docker compose -f docker-compose-nim-local.yaml ps

WARN[0000] /home/nvidia/cbrissette/retail-shopping-assistant/docker-compose-nim-local.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
NAME                                        IMAGE                                                             COMMAND                  SERVICE         CREATED         STATUS         PORTS
retail-shopping-assistant-content-1         nvcr.io/nim/nvidia/llama-3.1-nemoguard-8b-content-safety:1.10.1   "/opt/nvidia/nvidia_…"   content         9 minutes ago   Up 9 minutes   0.0.0.0:8003->8000/tcp, [::]:8003->8000/tcp
retail-shopping-assistant-embedqa-1         nvcr.io/nim/nvidia/nv-embedqa-e5-v5:1.6                           "/opt/nim/start_serv…"   embedqa         9 minutes ago   Up 9 minutes   0.0.0.0:8001->8000/tcp, [::]:8001->8000/tcp
retail-shopping-assistant-llama-1           nvcr.io/nim/meta/llama-3.1-70b-instruct:latest                    "/opt/nvidia/nvidia_…"   llama           9 minutes ago

This command should produce similar output in the following format:

```
CONTAINER ID   NAMES                                   STATUS
1dd42caad60e   retail-shopping-assistant-llama-1      Up 10 minutes
766acb5fb57c   retail-shopping-assistant-nvclip-1     Up 10 minutes
4c4d1136cd7a   retail-shopping-assistant-embedqa-1    Up 10 minutes
ff2f71eb9d75   retail-shopping-assistant-content-1    Up 10 minutes
fd70635efcac   retail-shopping-assistant-topic_control-1 Up 10 minutes
```

## Cloud NIM Deployment

### Step 1: Set Configuration for Cloud NIMs

In [ ]:
# Set configuration override for cloud NIM deployment
os.environ["CONFIG_OVERRIDE"] = "config-build.yaml"
print(f"Configuration override set to: {os.environ['CONFIG_OVERRIDE']}")

If you don't have local GPU resources, you can use cloud-hosted NIM endpoints. The application will automatically use the NVIDIA API Catalog hosted endpoints when local NIMs are not available.

<div class="alert alert-block alert-info">
    <b>Note:</b> When using cloud NIM deployment, you don't need to start the local NIM containers. The application will use the NVIDIA API Catalog hosted endpoints.
</div>

## Application Deployment

Docker compose scripts are provided which spin up the microservices on a single node. This docker-compose yaml file will start the retail shopping assistant services as well as dependent microservices. This may take up to **10 minutes** to complete.

<div class="alert alert-block alert-success">
    <b>Tip:</b> Refer to the docker-compose.yaml for complete details.
</div>

In [24]:
# Build and start application services
!cd .. && docker compose -f docker-compose.yaml up -d --build

WARN[0000] /home/nvidia/cbrissette/retail-shopping-assistant/docker-compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
Compose can now delegate builds to bake for better performance.
 To do so, set COMPOSE_BAKE=true.


[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (3/4)                                          docker:default
 => [rails internal] load build definition from Dockerfile                 0.0s
 => => transferring dockerfile: 534B                                       0.0s
 => [memory-retriever internal] load build definition from Dockerfile      0.0s
 => => transferring dockerfile: 380B                                       0.0s
 => [catalog-retriever internal] load build definition from Dockerfile     0.0s
 => => transferring dockerfile: 317B                                       0.0s
 => [memory-retriever internal] load metadata for docker.io/library/pytho  0.2s
[+] Building 0.3s (3/4)                                          docker:default
 => [rails internal] load build definition from Dockerfile                 0.0s
 => => transferring dockerfile: 534B                                       0.0s
 => [memory-retriever internal] load bui

<div class="alert alert-block alert-success">
    <b>Tip:</b> If you would like to monitor progress, refer to https://docs.docker.com/reference/cli/docker/compose/logs/.
</div>

In [25]:
# Monitor application startup
!cd .. && timeout 60 docker logs -f chain-server

INFO:     Will watch for changes in these directories: ['/app']
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)
INFO:     Started reloader process [1] using StatReload
2025-07-24 13:52:23,592 - WARNING - Override config file not found at /app/shared/configs/chain_server/config-local.yaml
2025-07-24 13:52:23,592 - INFO - PlannerAgent.__init__() | llm_name=meta/llama-3.1-70b-instruct, llm_port=http://localhost:8000/v1
2025-07-24 13:52:23,619 - INFO - PlannerAgent.__init__() | initialization complete
2025-07-24 13:52:23,619 - INFO - RetrieverAgent.__init__() | Initializing with llm_name=meta/llama-3.1-70b-instruct, llm_port=http://localhost:8000/v1
2025-07-24 13:52:23,641 - INFO - RetrieverAgent.__init__() | Initialization complete
2025-07-24 13:52:23,641 - INFO - CartAgent.__init__() | Initializing with llm_name=meta/llama-3.1-70b-instruct, llm_port=http://localhost:8000/v1
2025-07-24 13:52:23,662 - INFO - CartAgent.__init__() | Initialization complete
2025-07-24 

To validate the deployment of the blueprint, execute the following command to ensure the containers are running.

In [26]:
!cd .. && docker compose -f docker-compose.yaml ps

WARN[0000] /home/nvidia/cbrissette/retail-shopping-assistant/docker-compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
NAME                                        IMAGE                                                             COMMAND                  SERVICE             CREATED          STATUS                        PORTS
catalog-retriever                           retail-shopping-assistant-catalog-retriever                       "uvicorn app.main:ap…"   catalog-retriever   2 minutes ago    Up About a minute             
chain-server                                retail-shopping-assistant-chain-server                            "uvicorn app.main:ap…"   chain-server        2 minutes ago    Up About a minute             
memory-retriever                            retail-shopping-assistant-memory-retriever                        "uvicorn app.main:ap…"   memory-retriever    2 minutes ago    Up About a minute           

This command should produce similar output in the following format:

```
CONTAINER ID   NAMES                       STATUS
e6e1f6ebec3c   shopping-frontend           Up 3 hours
b6a1853c4e81   chain-server                Up 3 hours
91487a937be1   catalog-retriever           Up 3 hours
0112183489fe   memory-retriever            Up 3 hours
9970bb569dbd   rails                       Up 3 hours
4ea1a3267a17   milvus-standalone           Up 3 hours
c988dcdd67c3   milvus-minio                Up 3 hours (healthy)
3dc1c2262903   milvus-etcd                 Up 3 hours (healthy)
```

<div class="alert alert-block alert-info">
    <b>Note:</b> The NIM microservices are not listed since they are either running locally (if using local deployment) or using hosted endpoints (if using cloud deployment).
</div>

## Validate Deployment

The blueprint includes a reference UI and an AI assistant (developed using the LangGraph framework) that leverages sub-agents to handle retail shopping queries. Let's make sure the API endpoints and UI are up and running.

### 1. Test our server health.

In [27]:
# Test the chain-server health
!curl -s http://localhost:8009/health | python -m json.tool

{
    "status": "healthy",
    "timestamp": 1753365256.9792936,
    "version": "1.0.0"
}


In [28]:
# Test the retriever health
!curl -s http://localhost:8010/health | python -m json.tool

{
    "status": "healthy",
    "timestamp": 1753365259.9945495,
    "version": "1.0.0"
}


In [29]:
# Test the memory server health
!curl -s http://localhost:8011/health | python -m json.tool

{
    "status": "healthy",
    "timestamp": 1753365262.3632474,
    "version": "1.0.0"
}


### 2. Access the Web UI

Open your browser and navigate to: `http://localhost:3000`

<div class="alert alert-block alert-info">
    <b>Note:</b> If you are using an environment deployed with Brev, make sure to expose port 3000 on your Brev console and use the HTTP URL created.
</div>

## API Reference

For detailed API references, please refer to the following locations in the Blueprint repository:

- **Chain Server API**: `./docs/API.md`
- **User Guide**: `./docs/USER_GUIDE.md`

## Next Steps

Now that your Retail Shopping Assistant is deployed, you can:

1. **Explore the UI**: Test the conversational interface, product search, and cart management features
2. **Customize the Application**: Modify the configuration files to adapt to your specific retail domain
3. **Add Your Data**: Replace the sample data with your own product catalog and user data
4. **Scale the Deployment**: Consider production deployment options for high availability
5. **Monitor Performance**: Use the built-in monitoring and analytics features

For detailed customization and development guides, see:
- [User Guide](docs/USER_GUIDE.md)
- [API Documentation](docs/API.md)
- [Deployment Guide](docs/DEPLOYMENT.md)

## Stopping Services and Cleaning Up

To shut down the microservices, run the following commands:

In [ ]:
# Stop application services
!cd .. && docker compose -f docker-compose.yaml down

In [ ]:
# Stop local NIMs (if using local deployment)
!cd .. && docker compose -f docker-compose-nim-local.yaml down

## Appendix

### Switching Between Configurations

To switch between different configurations without redeploying:

```bash
# Switch to local NIMs
export CONFIG_OVERRIDE=config-local.yaml
docker compose -f docker-compose.yaml restart

# Switch to cloud NIMs
export CONFIG_OVERRIDE=config-build.yaml
docker compose -f docker-compose.yaml restart

# Use base configuration (no override)
unset CONFIG_OVERRIDE
docker compose -f docker-compose.yaml restart
```

### Creating Custom Configuration Overrides

You can create your own override files for custom configurations:

1. **Create the override file** in the same directory as the base config:
   ```bash
   # For chain server
   cp chain_server/app/config.yaml chain_server/app/config-custom.yaml
   
   # For catalog retriever
   cp catalog_retriever/src/config.yaml catalog_retriever/src/config-custom.yaml
   ```

2. **Modify the override file** with your custom values:
   ```yaml
   # Example: Custom LLM endpoint
   llm_port: "https://your-custom-endpoint.com/v1"
   llm_name: "your-custom-model"
   
   # Example: Custom embedding endpoint
   text_embed_port: "https://your-embedding-service.com/v1"
   text_model_name: "your-embedding-model"
   ```

3. **Use the custom override**:
   ```bash
   export CONFIG_OVERRIDE=config-custom.yaml
   docker compose -f docker-compose.yaml up -d --build
   ```

### Troubleshooting Common Issues

#### GPU Memory Issues
If you encounter GPU memory issues with local NIM deployment:

```bash
# Check GPU memory usage
nvidia-smi

# Adjust GPU allocation in docker-compose-nim-local.yaml
# Modify the device_ids for each service
```

#### Port Conflicts
If ports are already in use:

```bash
# Check what's using the ports
sudo netstat -tulpn | grep :8000
sudo netstat -tulpn | grep :3000

# Stop conflicting services or modify ports in docker-compose.yaml
```

#### NIM Startup Issues
If NIMs fail to start:

```bash
# Check NIM logs
docker compose -f docker-compose-nim-local.yaml logs -f

# Verify NGC API key
echo $NGC_API_KEY

# Check disk space
df -h
```

#### Configuration Override Issues
If configuration overrides are not working:

```bash
# Check if override file exists
ls -la chain_server/app/config-*.yaml
ls -la catalog_retriever/src/config-*.yaml

# Verify environment variable is set
echo $CONFIG_OVERRIDE

# Check application logs for config loading messages
docker compose -f docker-compose.yaml logs chain-server | grep -i config
```

### Performance Optimization

#### GPU Allocation
By default, GPU IDs are allocated as follows:
- **GPUs 0-1**: Llama 3.1 70B Instruct (LLM)
- **GPU 2**: NV-CLIP (Visual), NV-EmbedQA (Embeddings), Content Safety
- **GPU 3**: Topic Control

To change GPU allocation, modify the `device_ids` in `docker-compose-nim-local.yaml`:

```yaml
deploy:
  resources:
    reservations:
      devices:
        - driver: nvidia
          device_ids: ['0', '1']  # Change these IDs
          capabilities: [gpu]
```

#### Memory Optimization
For systems with limited RAM:

```yaml
# Add memory limits to docker-compose.yaml
services:
  chain-server:
    deploy:
      resources:
        limits:
          memory: 8G
```

### Production Deployment Considerations

For production deployments, consider:

1. **Load Balancing**: Use a reverse proxy (nginx, traefik) for high availability
2. **Monitoring**: Implement Prometheus/Grafana for metrics and alerting
3. **Logging**: Centralized logging with ELK stack or similar
4. **Security**: Implement proper authentication and authorization
5. **Backup**: Regular backups of Milvus and PostgreSQL data
6. **Scaling**: Horizontal scaling with Kubernetes for high traffic

See the [Deployment Guide](docs/DEPLOYMENT.md) for detailed production setup instructions.